In [46]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from pathlib import Path
import scipy.sparse as sparse
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.data import Dataset

In [47]:
df_sub = pd.read_csv(Path.cwd() / 'data' / 'sample_submission.csv')
df_train = pd.read_csv(Path.cwd() / 'data' / 'train.csv')
df_test = pd.read_csv(Path.cwd() / 'data' / 'test.csv')

/home/alex/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/alex/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [48]:
NUM_THREADS = 8 #число потоков
NUM_COMPONENTS = 60 #число параметров вектора 
NUM_EPOCHS = 25 #число эпох обучения

In [49]:
df_train.drop_duplicates(inplace=True)

In [50]:
df_train_train, df_train_test = train_test_split(df_train, random_state=32, shuffle=True)

In [51]:
df_train_train.columns

Index(['overall', 'verified', 'reviewTime', 'asin', 'reviewerName',
       'reviewText', 'summary', 'unixReviewTime', 'vote', 'style', 'image',
       'userid', 'itemid', 'rating'],
      dtype='object')

In [52]:
dataset = Dataset()

In [36]:
#Создаем датасет с размерностью макс. юзер на макс. итем.
dataset.fit(
            users=range(df_train.userid.max()+1),
            items=range(df_train.itemid.max()+1),
            user_features=['overall:1', 'overall:2', 'overall:3', 'overall:4', 'overall:5'],
            item_features=[]
)

In [37]:
dataset.model_dimensions()

(127501, 41320)

In [34]:
df_train_train[:40].apply(lambda x: (x.userid, {'overall': x.overall}), axis=1)

728049      (3215, {'overall': 5.0})
93056      (21567, {'overall': 5.0})
175063     (29578, {'overall': 4.0})
232990     (95970, {'overall': 3.0})
573251     (11032, {'overall': 1.0})
739522     (76177, {'overall': 5.0})
182507     (88504, {'overall': 5.0})
170523     (68500, {'overall': 5.0})
344858    (120023, {'overall': 4.0})
819926     (33541, {'overall': 5.0})
496273     (44977, {'overall': 3.0})
443253    (118067, {'overall': 5.0})
790734     (67214, {'overall': 5.0})
630639    (121594, {'overall': 5.0})
320707     (96667, {'overall': 2.0})
31031      (18000, {'overall': 5.0})
485819     (21423, {'overall': 5.0})
8142      (110903, {'overall': 5.0})
362969     (94511, {'overall': 5.0})
818566       (245, {'overall': 5.0})
479743     (23474, {'overall': 3.0})
255676     (17244, {'overall': 5.0})
553956     (63182, {'overall': 5.0})
466166     (54401, {'overall': 2.0})
322128     (32155, {'overall': 5.0})
293500    (110731, {'overall': 3.0})
435428      (2840, {'overall': 1.0})
6

In [31]:
df_train_train.iloc[749]

overall                                                           4
verified                                                       True
reviewTime                                              05 31, 2013
asin                                                     B001HTIUDC
reviewerName                                            GirlCopInFL
reviewText        Tasted different but tasted great!  Even used ...
summary                                                    LOVE IT!
unixReviewTime                                           1369958400
vote                                                              2
style                    {'Size:': ' 6 Jars', 'Flavor:': ' Creamy'}
image                                                           NaN
userid                                                        99540
itemid                                                         8701
rating                                                            1
Name: 177371, dtype: object

In [14]:
{b: x}

{0: 1}

Создаем матрицу взаимодействий на тренировочной части

In [38]:
(interactions, weights) = dataset.build_interactions(
    [(x[11], x[12], x[13]) for x in df_train_train.values])

In [39]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

In [44]:
len(user_feature_map)

127501

In [45]:
len(user_id_map)

127496

In [124]:
df_test[df_test.userid == 13457]

,verified,reviewTime,asin,reviewerName,unixReviewTime,vote,style,image,userid,itemid,Id
10,True,"03 8, 2014",B00778B90S,Ellie Woods,1394236800,7,{'Size:': ' 20 pack'},NaN,13457,18145,10
14706,True,"06 5, 2014",B0001LO3FG,Ellie Woods,1401926400,NaN,"{'Size:': ' 20 Count (Pack of 6)', 'Flavor:': ...",NaN,13457,379,14706
68091,True,"06 5, 2014",B000F4DKAI,Ellie Woods,1401926400,NaN,NaN,NaN,13457,1919,68091
187150,True,"06 5, 2014",B00XA8XWGS,Ellie Woods,1401926400,NaN,"{'Size:': ' 20 Count (Pack of 6)', 'Flavor:': ...",NaN,13457,33788,187150
222601,True,"06 12, 2013",B01HGBAFNC,Ellie Woods,1370995200,NaN,"{'Size:': ' 20 Count (Pack of 8)', 'Flavor:': ...",NaN,13457,38299,222601
247755,True,"06 12, 2013",B002HFRAF4,Ellie Woods,1370995200,NaN,"{'Size:': ' Pack of 8', 'Flavor:': ' Vanilla'}",NaN,13457,10821,247755


In [109]:
model = LightFM(learning_rate=0.1, loss='logistic')

In [110]:
model.fit(interactions, # spase matrix representing whether user u and item i interacted
      sample_weight= weights, # spase matrix representing how much value to give to user u and item i inetraction: i.e ratings
      epochs=10, num_threads=NUM_THREADS)

In [27]:
from lightfm.evaluation import auc_score
train_auc = auc_score(model,
                      interactions
                     ).mean()
print('Hybrid training set AUC: %s' % train_auc)

Hybrid training set AUC: 0.92077446


In [39]:
ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])))

In [40]:
model = LightFM(learning_rate=0.1, loss='logistic',
                no_components=NUM_COMPONENTS)
model = model.fit(ratings_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

In [34]:
len(df_sub)

285965

In [35]:
len(df_test)

285965

In [121]:
def cold_start(row, user_id_map, item_id_map, model):
    if row.userid in user_id_map and row.itemid in item_id_map:
        return model.predict(user_id_map[row.userid], [item_id_map[row.itemid]])[0]
    else:
        print(row.userid, row.itemid)
        return 0

In [112]:
model.predict(user_id_map[50442], [item_id_map[15530]])

array([1.04392183])

In [122]:
pred_set = df_test[0:1000].apply(cold_start, axis=1, user_id_map=user_id_map, 
                    item_id_map=item_id_map, model=model)

13457 18145
85892 37102
56698 28650
124621 33986
110673 12113
37076 13307


In [44]:
preds_sub = model.predict(df_test.userid.values,
                      df_test.itemid.values)

In [114]:
pred_set.min(), pred_set.max()

(-0.007606500294059515, 9.58047103881836)

In [115]:
normalized_preds = (pred_set - pred_set.min())/(pred_set - pred_set.min()).max()

In [116]:
normalized_preds.min(), normalized_preds.max()

(0.0, 1.0)

In [117]:
df_sub['rating']= normalized_preds

In [118]:
df_sub.to_csv('submission1_log.csv', index=False)